# AWS 사용자 개인화 알고리즘 및 사용자-항목-상호 작용 데이터를 사용한 배치 추천 <a class="anchor" id="top"></a>

이 노트북에서는 데이터 세트를 선택하고 Amazon Personalize 배치 추천에 사용할 수 있도록 준비합니다.

1. [데이터 세트 또는 데이터 소스 선택](#source)
1. [데이터 준비](#prepare)
1. [데이터 세트 그룹 및 상호 작용 데이터 세트 생성](#group_dataset)
1. [S3 버킷 및 IAM 역할 구성](#bucket_role)
1. [상호 작용 데이터 가져오기](#import)
1. [솔루션 생성](#solutions)
1. [배치 추천](#batch)
1. [정리](#cleanup)

## 소개 <a class="anchor" id="intro"></a>

대부분의 경우, Amazon Personalize(레시피라고 함)의 알고리즘은 여기서 설명하는 다양한 작업을 처리합니다.

1. **AWS 사용자 개인화** - 이전 사용자 상호 작용을 기준으로 항목을 추천합니다.
1. **개인별 순위** - 항목 모음을 선택한 다음 HRNN과 유사한 접근 방식을 사용하여 항목 순서를 지정합니다.
1. **SIMS(유사 항목)** - 특정 항목이 주어지면 사용자가 상호 작용하는 다른 항목도 추천합니다.
1. **인기 카운트** - HRNN 또는 HRNN 메타데이터에서 적절한 정보를 찾을 수 없는 경우 가장 인기 있는 항목을 추천합니다. 이 항목은 기본적으로 반환됩니다.

사용 사례에 관계없이, 모든 알고리즘은 3가지 핵심 속성으로 정의된 사용자-항목-상호 작용 데이터에 대한 학습 기반을 공유합니다.

1. **UserID** - 상호 작용한 사용자
1. **ItemID** - 사용자가 상호 작용한 항목
1. **타임스탬프** - 상호 작용이 발생한 시간

또한 다음과 같이 정의된 이벤트 유형과 이벤트 값도 지원합니다.

1. **이벤트 유형** - 이벤트의 범주형 레이블(찾아보기, 구매, 평점 등)입니다.
1. **이벤트 값** - 발생한 이벤트 유형에 해당하는 값입니다. 간단히 말해, 이벤트 유형에 대해 0과 1 사이의 정규화된 값을 찾는 것입니다. 예를 들어 3단계(클릭, 장바구니에 추가, 구매)로 거래를 완료하는 경우 각 단계마다 event_value는 각각 0.33, 0.66, 1.0이 됩니다.

이벤트 유형 및 이벤트 값 필드는 개인화 모델 훈련을 위해 전송된 데이터를 필터링하는 데 사용할 수 있는 추가 데이터입니다. 이 연습에서는 이벤트 유형이나 이벤트 값이 없습니다.

## 데이터 세트 또는 데이터 소스 선택 <a class="anchor" id="source"></a>
[맨 위로 이동](#top)

앞서 언급했듯이, 사용자-항목-상호 작용 데이터는 서비스를 시작하기 위한 핵심 속성입니다. 즉, 이러한 종류의 데이터를 생성하는 사용 사례를 찾아야 합니다. 몇 가지 일반적인 예는 다음과 같습니다.

1. 비디오 온디맨드 애플리케이션
1. 전자 상거래 플랫폼
1. 소셜 미디어 집계기/플랫폼

Personalize에 적합한 문제의 범위를 지정하는 데 적용되는 몇 가지 지침이 있습니다. [공식 한도](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)는 조금 더 낮지만 아래의 값을 시작점으로 사용하면 유용합니다.

* 인증된 사용자
* 50명 이상의 고유한 사용자
* 100개 이상의 고유한 항목
* 사용자당 24건 이상의 상호 작용 

대부분의 경우 이는 쉽게 달성할 수 있으며 한 범주에서 값이 기준에 못 미치면 다른 범주에서 더 많은 수를 확보하여 보충할 수 있습니다.

일반적으로 데이터는 Personalize에 사용하기에 완벽한 형식으로 제공되지 않으며, 올바르게 구성되기 위해서는 약간의 수정이 필요합니다. 이 노트북에서는 그러한 모든 과정을 안내합니다.

먼저, [Last.FM](https://grouplens.org/datasets/hetrec-2011/) 데이터 세트를 사용해 보겠습니다. 이 데이터 세트는 사용자들의 음악 감상 행동에 대한 기록입니다. 이 데이터는 다수의 사용자, 항목 및 상호 작용에 대한 우리의 지침에 부합합니다.

먼저 아래의 코드를 사용하여 데이터 세트를 다운로드하고 새 폴더에 압축을 풉니다.

In [ ]:
data_dir = "data"
!mkdir $data_dir
!cd $data_dir && wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!cd $data_dir && unzip hetrec2011-lastfm-2k.zip

다운로드한 데이터 파일을 살펴봅니다.

In [ ]:
!ls $data_dir

현재 많은 .dat 파일과 README가 있다는 것 외에는 데이터에 대해 알려진 사실이 거의 없습니다. README를 열어보면 이 데이터의 전체 구조에 대해 알 수 있습니다. 데이터 소스가 외부 팀에서 제공되지 않는 한 사용자 지정 데이터를 사용하고 이 단계를 건너뛸 수 있습니다.

README에서, 이 데이터 세트에는 여러 가지 상호 작용 유형이 있음을 확인할 수 있습니다. 서로를 친구로 지정한 사용자 간의 상호 작용, 아티스트의 음악을 감상하는 사용자의 상호 작용, 사용자와 아티스트에 할당된 태그를 사용한 상호 작용 등입니다.

이 에에서는 사용자, 아티스트 및 음악 감상 상호 작용에 초점을 맞춥니다. 1,892명의 사용자, 17,632명의 아티스트(이 예에서 항목에 해당), 사용자가 아티스트의 음악을 감상하는 상호 작용 92,834건이 있습니다. 이 정도면 Personalize를 시작하는 데 충분합니다.

`Files` 섹션까지 README를 계속 읽습니다. 데이터 세트에 있는 대부분의 파일은 우리와 관련이 없지만, `users_artists.dat` 파일은 유용해 보입니다. README의 `Data format` 섹션에는 이 파일의 콘텐츠에 대한 세부 정보가 나와 있습니다. 여기서 첫 번째 문제에 부딪히게 됩니다.

| userID | artistID | weight  |
|--------|----------|---------|
| 2      | 51       | 13883   |

사용자와 사용자가 음악을 감상하는 아티스트 간의 상호 작용 데이터가 있지만, 상호 작용이 타임스탬프가 아니라 가중치로 저장되어 있습니다. Amazon Personalize에는 사용자-항목-타임스탬프 상호 작용 데이터가 필요합니다.

데이터 세트의 파일을 다시 살펴보면 `users_taggedartists-timestamps.dat`에 타임스탬프 데이터가 포함되어 있음을 알 수 있습니다. 음악 감상 행동 대신에 태깅 행동을 상호 작용 데이터로 사용한다면 어떨까요? 사용자가 아티스트를 태깅하는 행동을 긍정적인 감정의 표현으로 간주할 수 있을까요? 일반적으로, 해결하고자 하는 사용 사례에 이 상호 작용이 적합한지 여부를 이해하기 위해서는 고객 또는 도메인에 대해 잘 아는 사람과 논의해야 합니다. 여기서는 태깅 행동이 우리의 요구 사항에 적합하다고 가정하겠습니다.

`user_taggedartists-timestamps.dat`의 스키마는 다음과 같습니다.

| userID | artistID | tagID | timestamp     |
|--------|----------|-------|---------------|
| 2      | 52       | 13    | 1238536800000 |

`tagID` 속성을 제거하면 Amazon Personalize에 필요한 형식을 정확하게 얻을 수 있습니다.

## 데이터 준비 <a class="anchor" id="prepare"></a>
[맨 위로 이동](#top)

다음으로 데이터를 로드하고 데이터가 양호한 상태인지 확인한 다음, 데이터를 CSV에 저장하여 Amazon Personalize에 사용할 준비가 되도록 합니다.

시작하기 위해 데이터 과학에서 일반적으로 사용되는 Python 라이브러리 컬렉션을 가져옵니다.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime

import boto3
import pandas as pd

그런 다음 데이터 파일을 열고 처음 몇 행을 살펴봅니다.

In [ ]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat')
original_data.head(5)

보시다시피, 데이터가 올바르게 로드되지 않았습니다. CSV(쉼표로 구분된 값) 파일의 기본 구분 기호는 쉼표(`,`)이지만 이 예에서는 파일이 탭(`\t`) 구분 기호를 사용하여 저장되었습니다. 따라서 올바른 구분 기호를 지정하고 데이터를 다시 로드해 보겠습니다.

In [ ]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat', delimiter='\t')
original_data.head(5)

이제 제대로 됐습니다. 이제 데이터가 메모리에 성공적으로 로드되었으므로 몇 가지 추가 정보를 추출하겠습니다. 먼저, 데이터에서 몇 가지 기본 통계를 계산합니다.

In [ ]:
original_data.describe()

이는 `userID` 및 `artistID` 값의 범위가 충분하다는 것을 보여줍니다. 다음으로, 항상 데이터 형식을 확인하는 것이 좋습니다.

In [ ]:
original_data.info()

이와 관련해, 데이터 세트에 4개의 열에 걸쳐 총 186,479개의 항목이 있으며 각 셀은 int64 형식으로 저장되다는 것을 알 수 있습니다.

int64 형식은 확실히 `userID`와 `artistID`에 적합합니다. 하지만 좀 더 자세히 들어가서 데이터의 타임스탬프를 이해할 필요가 있습니다. Amazon Personalize를 사용하려면 타임스탬프를 [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) 형식으로 저장해야 합니다.

현재 이 타임스탬프 값은 사람이 읽을 수 없습니다. 그럼 임의의 타임스탬프 값을 가지고 해석하는 방법을 알아보겠습니다.

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

이런! 이 타임스탬프 값의 경우 코드는 41,132년이라는 연도를 렌더링했습니다. 이는 너무 먼 미래이므로, 데이터를 해석하는 방법이 잘못되었다는 것을 확실히 알 수 있습니다. 다시 시도해 보아야 합니다.

JavaScript는 시간을 밀리초 단위로 기록하며, 이 데이터 세트는 웹 애플리케이션에서 데이터를 수집한 것이므로 코드를 적용하기 전에 타임스탬프 값을 1000으로 나누겠습니다.

In [ ]:
arb_time_stamp = arb_time_stamp/1000
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

2009년 2월이라는 값은 이 데이터 세트에 대해 훨씬 더 현실적으로 느껴집니다. Amazon Personalize를 사용하는 데 사람이 읽을 수 있는 타임스탬프는 필요 없지만, 날짜를 사실적으로 표시해야 하므로 이제 데이터 세트의 각 타임스탬프를 JavaScript 밀리초 형식 이외의 형식으로 변환합니다. 

In [ ]:
original_data.timestamp = original_data.timestamp / 1000
original_data.head(5)

임의의 타임스탬프를 선택하고 사람이 읽을 수 있는 형식으로 변환하여 변환된 데이터 세트에 대해 간단한 온전성 검사를 수행합니다.

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

이 날짜는 타임스탬프로 적절하므로, 나머지 데이터의 형식을 계속 변환할 수 있습니다. 필요한 데이터는 사용자-항목-상호 작용 데이터이며, 이 예에서는 `userID`, `artistID` 및 `timestamp`입니다. 이 데이터 세트에는 데이터 세트에서 삭제할 수 있는 추가 열인 `tagID`가 있습니다.

In [ ]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['userID', 'artistID', 'timestamp']]
interactions_df.head()

데이터를 조작한 후에는 항상 데이터 형식이 변경되었는지 확인합니다.

In [ ]:
interactions_df.dtypes

이 예에서는 타임스탬프 열이 int64에서 float64로 변경되었습니다. 이제 형식을 int64로 다시 변경하겠습니다.

In [ ]:
interactions_df.astype({'timestamp': 'int64'}).dtypes

 Amazon Personalize에는 사용자, 항목 및 타임스탬프의 기본 열 이름이 있습니다. 이러한 기본 열 이름은 `USER_ID`, `ITEM_ID` 및 `TIMESTAMP`입니다. 따라서 데이터 세트의 마지막 수정 사항은 기존 열 헤더를 기본 헤더로 바꾸는 것입니다.

In [ ]:
interactions_df.rename(columns = {'userID':'USER_ID', 'artistID':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

이제 끝났습니다. 이제 데이터가 사용 가능한 상태로 준비되었으므로, CSV 파일로 저장하기만 하면 됩니다.

In [ ]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## 데이터 세트 그룹 및 상호 작용 데이터 세트 생성 <a class="anchor" id="group_dataset"></a>
[맨 위로 이동](#top)

Amazon Personalize에서 격리 및 추상화의 최상위 수준은 데이터 세트 그룹입니다. 이러한 데이터 세트 그룹 중 하나에 저장된 정보는 다른 데이터 세트 그룹이나 거기에서 생성된 모델에 영향을 주지 않으며 완전히 격리됩니다. 이를 통해 많은 실험을 실행할 수 있으며, 이는 모델을 비공개 상태로 사용자의 데이터에 대해서만 완벽하게 훈련된 상태로 유지하기 위한 방법 중 하나입니다.

이전에 준비된 데이터를 가져오기 전에, 데이터 세트 그룹이 필요하고 해당 데이터 세트 그룹에 상호 작용을 처리하는 데이터 세트를 추가해야 합니다.

데이터 세트 그룹은 다음과 같은 유형의 정보를 저장할 수 있습니다.

* 사용자-항목-상호 작용
* 이벤트 스트림(실시간 상호 작용)
* 사용자 메타데이터
* 항목 메타데이터

상호 작용 데이터의 데이터 세트 그룹과 데이터 세트를 만들기 전에, 사용자 환경이 Amazon Personalize와 정상적으로 통신할 수 있는지 검증해 보겠습니다.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### 데이터 세트 그룹 생성

다음 셀은 `personalize-poc-lastfm`이라는 이름으로 새 데이터 세트 그룹을 만듭니다.

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-batch-recommendations-dg"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

데이터 세트 그룹을 사용하려면 먼저 그룹이 활성화되어 있어야 합니다. 활성화하는 데에는 1~2분이 걸립니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 3시간 동안 매초마다 데이터 세트 그룹의 상태를 확인합니다.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

이제 데이터 세트 그룹을 만들었으므로 상호 작용 데이터의 데이터 세트를 만들 수 있습니다.

### 데이터 세트 생성

먼저 업로드하려는 데이터 세트의 유형을 Amazon Personalize에 알려주는 스키마를 정의합니다. 스키마에는 데이터 세트 유형에 따라 몇 가지 예약 키워드와 필수 키워드가 필요합니다. 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)를 참조하세요.

여기서는 `USER_ID`, `ITEM_ID` 및 `TIMESTAMP` 필드가 필요한 상호 작용 데이터에 대한 스키마를 만듭니다. 데이터 세트에서 나타나는 순서와 동일한 순서로 스키마에 정의해야 합니다.

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-batch-recommendations-interactions",
    schema = json.dumps(interactions_schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

생성된 스키마를 사용하여 데이터 세트 그룹 내에 데이터 세트를 만들 수 있습니다. 데이터는 아직 로드하지 않는다는 점을 유의하세요. 데이터는 몇 단계 후에 로드합니다.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-batch-recommendations-ds",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## S3 버킷 및 IAM 역할 구성 <a class="anchor" id="bucket_role"></a>
[맨 위로 이동](#top)

지금까지 이 Jupyter 노트북을 실행하는 인스턴스에 연결된 Amazon EBS 인스턴스에 데이터를 다운로드하고 조작하고 저장했습니다. 하지만 Amazon Personalize에는 데이터의 소스 역할을 하는 S3 버킷과 해당 버킷에 액세스하기 위한 IAM 역할이 필요합니다. 이 버킷과 역할을 설정해보겠습니다.

이 Amazon SageMaker 노트북의 기반이 되는 인스턴스에 저장된 메타데이터를 사용하여 해당 노트북이 운영되고 있는 리전을 확인합니다. Amazon SageMaker 외부에서 Jupyter 노트북을 사용하는 경우 아래의 문자열로 리전을 정의하면 됩니다. Amazon S3 버킷은 지금까지 생성한 Amazon Personalize 리소스와 동일한 리전에 있어야 합니다.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Amazon S3 버킷 이름은 전역적으로 고유합니다. 고유한 버킷 이름을 만들기 위해 아래 코드는 AWS 계정 번호에 `personalizepoc` 문자열을 추가합니다. 그런 다음 이전 셀에서 확인된 리전에 이 이름으로 버킷을 만듭니다.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-personalize-batch-recommendations"
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### S3에 데이터 업로드

이제 Amazon S3 버킷이 생성되었으므로 사용자-항목-상호 작용 데이터의 CSV 파일을 업로드합니다. 

In [ ]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### S3 버킷 정책 설정
Amazon Personalize가 S3 버킷의 콘텐츠를 읽을 수 있어야 합니다. 이를 허용하는 버킷 정책을 추가합니다.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### IAM 역할 생성

Amazon Personalize가 특정 작업을 실행할 수 있는 권한을 가지려면, AWS에서 역할을 수임할 수 있어야 합니다. IAM 역할을 생성하고 필요한 정책을 연결해보겠습니다. 아래의 코드는 과도하게 권한을 부여하는 정책을 연결합니다. 프로덕션 애플리케이션에는 보다 제한적인 정책을 사용하세요.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleBatchRecommendations"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## 상호 작용 데이터 가져오기 <a class="anchor" id="import"></a>
[맨 위로 이동](#top)

앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 S3 버킷에서 Amazon Personalize 데이터 세트로 데이터를 로드하는 가져오기 작업을 실행합니다. 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-batch-recommendations",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

데이터 세트를 사용하려면 먼저 가져오기 작업이 활성 상태여야 합니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 3시간 동안 매초마다 가져오기 작업의 상태를 확인합니다.

데이터 세트의 크기에 따라 데이터를 가져오는 데 시간이 걸릴 수 있습니다. 이 워크숍에서는 데이터 가져오기 작업에 약 15분이 소요됩니다.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

데이터 세트 가져오기가 활성화되면 SIMS, 개인별 순위, 인기 카운트, AWS 사용자 개인화를 사용하여 모델을 구축할 준비가 된 것입니다. 이 프로세스는 다른 노트북에서도 계속됩니다. 다음 셀을 실행하여 다음 노트북에 사용할 몇 가지 값을 저장하세요.

## 솔루션 생성 <a class="anchor" id="solutions"></a>
[맨 위로 이동](#top)

이 노트북에서는 다음 레시피로 솔루션을 만듭니다.

1. AWS 사용자 개인화


Amazon Personalize에서는 알고리즘의 특정 변형 버전을 레시피라고 합니다. 레시피마다 적용되는 상황이 다릅니다. 훈련된 모델을 솔루션이라고 하며, 각 솔루션은 모델을 훈련할 때 특정 데이터 볼륨과 관련된 여러 버전을 가질 수 있습니다.

먼저 지원되는 모든 레시피를 나열하겠습니다. 이렇게 하면 모델을 선택하고 모델을 구축하는 데 사용할 수 있습니다.

In [ ]:
personalize.list_recipes()

출력은 도입부에 언급된 모든 알고리즘의 JSON 표현일 뿐입니다.

다음으로 구체적인 레시피를 선택하여 그 레시피로 모델을 만듭니다.

### AWS 사용자 개인화

AWS 사용자 개인화는 여러분이 사용할 수 있는 고급 추천 모델 중 하나이며 사용자 행동에 따라 추천을 실시간으로 업데이트할 수 있습니다. 또한 협업 필터링과 같은 다른 접근 방식을 능가하는 경우가 많습니다. 이 레시피는 훈련 시간이 가장 오래 걸리는데, 우선 이 레시피부터 시작하겠습니다.

이 사용 사례의 경우 LastFM 데이터를 사용합니다. AWS 사용자 개인화 알고리즘을 사용하여 사용자의 이전 아티스트 태깅 행동에 따라 새로운 아티스트를 사용자에게 추천할 수 있습니다. 태깅 데이터를 사용하여 사용자와 아티스트 간의 긍정적인 상호 작용을 표현했다는 것을 기억하실 겁니다.

먼저 위의 레시피 목록에서 ARN을 찾아 레시피를 선택합니다.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### 솔루션 생성

먼저 레시피를 사용하여 솔루션을 만듭니다. 이 단계에서 데이터 세트 ARN을 제공하지만 모델은 아직 훈련되지 않았습니다. 따라서 훈련된 모델이 아니라 식별자로 생각해야 합니다.

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-batch-recommendations-user-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### 솔루션 버전 생성

솔루션을 찾은 후에는 모델 훈련을 완료하기 위해 버전을 만들어야 합니다. 훈련을 완료하는 데 25분 이상 걸릴 수 있으며, 이 예의 데이터 세트를 사용하여 이 레시피를 완료하는 데 평균 40분이 소요됩니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 많은 모델을 만들고 신속하게 배포하는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 솔루션에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

In [ ]:
solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

### 솔루션 생성 상태 보기

예고해 드린 대로, 콘솔에서 상태 업데이트를 보는 방법은 다음과 같습니다.

* 다른 브라우저 탭에서 이 노트북 인스턴스를 열 때 AWS 콘솔이 이미 열려 있어야 합니다.
* 해당 탭으로 전환하고 맨 위에서 `Personalize` 서비스를 검색한 다음 해당 서비스 페이지로 이동합니다.
* `View dataset groups`를 클릭합니다.
* 데이터 세트 그룹의 이름을 클릭합니다. 이름에 POC가 있을 가능성이 높습니다.
* `Solutions and recipes`를 클릭합니다.
* 이제 솔루션 버전 상태가 표시된 열을 포함하여, 위에서 생성한 모든 솔루션 목록이 표시됩니다. `Active` 상태가 되면 솔루션을 검토할 준비가 된 것입니다. 또한 배포할 수도 있습니다.

또는 아래의 셀을 실행하여 솔루션 버전 생성 상태를 추적합니다.

In [ ]:
in_progress_solution_versions = [
    solution_version_arn,
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

## 솔루션과 상호 작용 <a class="anchor" id="interact"></a>
[맨 위로 이동](#top)

이제 모든 솔루션이 배포되고 활성화되었으므로 API 호출을 통해 추천을 받을 수 있습니다.

먼저 조회 테이블에 사용할 수 있는 데이터 세트를 로드합니다.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

ID 열을 인덱스 열로 정의하면 ID를 쿼리하는 것만으로 아티스트를 반환할 수 있습니다.

In [ ]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

그렇게 나쁘지는 않지만, 코드의 모든 곳에서 이 작업을 반복하면 더 복잡해집니다. 그래서 아래의 함수가 이를 정리해줄 것입니다.

In [ ]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

이제 오류 캐칭 기능을 확인하기 위해 몇 가지 간단한 값을 테스트해 보겠습니다.

In [ ]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

좋습니다. 이제 결과를 렌더링할 수 있습니다. 

## 배치 추천 <a class="anchor" id="batch"></a>
[맨 위로 이동](#top)

내보낸 추천으로 이루어진 더 큰 데이터 세트가 필요한 경우가 많습니다. 최근 Amazon Personalize는 추천 컬렉션을 S3로 내보내는 수단으로서 배치 추천을 발표했습니다. 이 예에서는 AWS 사용자 개인화 솔루션에 대해 이 작업을 수행하는 방법을 살펴보겠습니다. 배치 추천에 대한 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch)를 참조하세요. 이 기능은 모든 레시피에 적용되지만 출력 형식은 레시피마다 다릅니다.

간단한 구현의 예는 다음과 같습니다.

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

SDK 가져오기, 솔루션 버전 알림 및 역할 알림이 모두 결정되었습니다. 이제 입력, 출력 및 작업 이름만 정의하면 됩니다.

사용자 개인화에 대한 입력부터 시작하겠습니다. 이 입력은 다음과 같습니다.


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

이 경우 다음과 같은 출력이 생성됩니다.

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

출력은 JSON Lines 파일로, 한 줄에 하나씩 개별 JSON 객체로 구성됩니다. 따라서 나중에 결과를 이 형식으로 수집하기 위해서는 추가 작업이 필요합니다.

### 입력 파일 생성

배치 기능을 사용할 경우, 작업이 완료되었을 때 추천을 받을 사용자를 지정합니다. 아래의 셀은 다시 몇 명의 임의의 사용자를 선택한 후 파일을 만들고 디스크에 저장합니다. 그런 다음 S3에 업로드하여 나중에 API 호출에 사용할 수 있습니다.

In [ ]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

In [ ]:
# Get the user list
batch_users = users_df.sample(3).index.tolist()

# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [ ]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

파일을 S3에 업로드하고 나중에 사용할 수 있도록 경로를 변수로 저장합니다.

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

배치 추천은 S3에 업로드한 파일에서 입력 데이터를 읽습니다. 마찬가지로 배치 추천은 출력을 S3의 파일에 저장합니다. 따라서 결과를 저장할 출력 경로를 정의합니다.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

이제 배치 내보내기 프로세스를 시작하는 호출을 실행하면 됩니다.

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "Batch-Inference-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

배치 추천 호출의 상태를 추적하기 위해 아래의 while 루프를 실행합니다. Personalize는 이 작업을 수행하기 위해 인프라를 구축해야 하므로, 작업을 완료하는 데 25분 정도 걸릴 수 있습니다. 여기서는 3명의 사용자로 이루어진 데이터 세트로 이 기능을 테스트하고 있는데, 이는 이 메커니즘의 효율적인 사용 사례라고 할 수 없습니다. 일반적으로 이 기능은 대량 처리에만 사용되며, 이 경우 높은 효율성이 명확하게 나타납니다.

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

배치 추천 작업의 처리가 완료되면 S3에 업로드된 출력을 가져와 구문 분석할 수 있습니다.

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            artist = get_artist_by_id(item)
            recommendation_list.append(artist)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## 솔루션 정리

다음으로 솔루션을 정리합니다. 아래 코드는 계정에 있는 모든 솔루션을 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

ARN 목록을 확인하여 삭제할 솔루션을 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 삭제합니다.

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## 데이터 세트 정리

다음으로 데이터 세트를 정리합니다. 아래 코드는 계정에 있는 모든 데이터 세트를 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

ARN 목록을 확인하여 삭제할 데이터 세트를 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 삭제합니다.

In [ ]:
personalize.delete_dataset(
    datasetArn = "INSERT ARN HERE"
)

## 스키마 정리

다음으로 스키마를 정리합니다. 아래 코드는 계정에 있는 모든 스키마를 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

ARN 목록을 확인하여 삭제할 스키마를 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 삭제합니다.

In [ ]:
personalize.delete_schema(
    schemaArn = "INSERT ARN HERE"
)

## 데이터 세트 그룹 정리

마지막으로, 데이터 세트 그룹을 정리합니다. 아래 코드는 계정에 있는 모든 데이터 세트 그룹을 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

ARN 목록을 확인하여 삭제할 데이터 세트 그룹을 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 삭제합니다.

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = "INSERT ARN HERE"
)

## S3 버킷 및 IAM 역할 정리

원하는 경우 IAM 역할과 워크숍을 진행하는 동안 생성한 S3 버킷을 삭제할 수 있습니다.

아래 코드를 사용하여 계정의 모든 IAM 역할을 나열하는 것부터 시작합니다.

In [ ]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

삭제할 역할의 이름을 식별합니다.

아직 정책이 연결되어 있는 IAM 역할은 삭제할 수 없습니다. 따라서 관련 역할을 식별한 후, 해당 역할에 연결된 정책을 나열해 봅니다.

In [ ]:
iam.list_attached_role_policies(
    RoleName = "INSERT ROLE NAME HERE"
)

아래 코드를 사용하여 위의 결과에서 정책을 분리해야 합니다. 연결된 정책마다 이 작업을 반복합니다.

In [ ]:
iam.detach_role_policy(
    RoleName = "INSERT ROLE NAME HERE",
    PolicyArn = "INSERT ARN HERE"
)

이제 IAM 역할을 삭제할 수 있습니다.

In [ ]:
iam.delete_role(
    RoleName = "INSERT ROLE NAME HERE"
)

S3 버킷을 삭제하려면 먼저 버킷이 비어 있어야 합니다. S3 버킷을 삭제하는 가장 쉬운 방법은 AWS 콘솔에서 S3으로 이동하여 버킷의 객체를 삭제한 다음 S3 버킷 자체를 삭제하는 것입니다.